In [ ]:
%pip install pytesseract

In [ ]:
import torch
print(f"Metal Performance Shaders (MPS) is {torch.backends.mps.is_available()}")

In [1]:
from collections import defaultdict
import cv2
import numpy as np
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO("yolov10b.pt")

# Open the video file
video_path = "test.mp4"
cap = cv2.VideoCapture(video_path)

# Store the track history
track_history = defaultdict(lambda: [])

# Define movement threshold (you can adjust this value)
movement_threshold = 5  # pixels

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True, device='mps')

        # Get the boxes and track IDs
        boxes = results[0].boxes.xywh.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = track_history[track_id]
            if len(track) > 0:
                # Calculate the movement
                last_x, last_y = track[-1]
                movement = np.sqrt((x - last_x)**2 + (y - last_y)**2)

                # Check if the car is moving or stopped
                if movement < movement_threshold:
                    status = "Stopped"
                    color = (0, 0, 255)  # Red color for stopped
                else:
                    status = "Moving"
                    color = (0, 255, 0)  # Green color for moving

                # Display status on the frame
                cv2.putText(annotated_frame, status, (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Update the track history
            track.append((float(x), float(y)))  # x, y center point
            if len(track) > 30:  # retain 30 tracks for recent frames
                track.pop(0)

            # Draw the tracking lines
            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=2)

        # Display the annotated frame
        cv2.imshow("YOLOv8 Tracking", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()



0: 384x640 2 cars, 499.9ms
Speed: 41.5ms preprocess, 499.9ms inference, 434.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 60.2ms
Speed: 2.5ms preprocess, 60.2ms inference, 11.2ms postprocess per image at shape (1, 3, 384, 640)



2024-08-13 18:43:50.205 Python[36579:1001015] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


0: 384x640 2 cars, 46.7ms
Speed: 2.1ms preprocess, 46.7ms inference, 15.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 45.1ms
Speed: 2.3ms preprocess, 45.1ms inference, 14.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 45.2ms
Speed: 2.0ms preprocess, 45.2ms inference, 15.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 44.9ms
Speed: 2.3ms preprocess, 44.9ms inference, 15.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 48.9ms
Speed: 2.0ms preprocess, 48.9ms inference, 58.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 51.0ms
Speed: 2.4ms preprocess, 51.0ms inference, 14.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 44.8ms
Speed: 2.6ms preprocess, 44.8ms inference, 14.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 45.3ms
Speed: 2.1ms preprocess, 45.3ms inference, 15.2ms postprocess per image at shape (1, 3, 384, 640)



: 